In [231]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from bertopic import BERTopic

from sentence_transformers import SentenceTransformer
from nltk import pos_tag
import sklearn.manifold
import random
random.seed(42)

import yaml
from yaml.loader import SafeLoader
import datetime
import re
import sys
sys.path.append('./mylib')
#from sentiment_process import *
import global_materials

stopwords = global_materials.stopwords
exclude_stopwords = {"app"}
stopwords -= exclude_stopwords

In [232]:
# PARAMETER
#TOPIC_MODEL_PATH = './model/release_2022_10_03/'
TOPIC_MODEL_PATH = './model/dev_2022_10_04/'
DATE_FROM = datetime.datetime(2022,9,1)
DATE_TO = datetime.datetime(2022,9,30)
REVIEW_FILE_PATH = "../sentiment_analysis_external_review_nbank/data/predict/OUT_sentiment_prediction.csv"



In [233]:
PRED_ROOT = './predict_data/'

if not os.path.exists("model"):
    os.mkdir("model")
if not os.path.exists(PRED_ROOT):
    os.mkdir(PRED_ROOT)

In [234]:
# # External review data
# file = "../sentiment_analysis_external_review/data/train/5_dataApp_EN_clean_vader.csv"
# #file = "4_dataApp_EN_2022-09-23.csv"
# df_pred_ex = pd.read_csv(file, index_col=False)
# df_pred_ex = df_pred_ex[['date','review','review_adj','review_en','label']]
# df_pred_ex['date'] = pd.to_datetime(df_pred_ex['date'])
# df_pred_ex = df_pred_ex[df_pred_ex['date']>=datetime.datetime(2022,9,1)]

# # Internal review data
# file = "../sentiment_analysis_internal_review/data/train/5_dataApp_EN_clean_vader.csv"
# #file = "4_dataApp_EN_2022-09-23.csv"
# df_pred_in = pd.read_csv(file, index_col=False)
# df_pred_in = df_pred_in[['date','review','review_adj','review_en','label']]
# df_pred_in['date'] = pd.to_datetime(df_pred_in['date'])
# df_pred_in = df_pred_in[df_pred_in['date']>=datetime.datetime(2022,9,1)]

# df_pred = pd.concat([df_pred_ex, df_pred_in])
# print(f'Total review in this month is: {df_pred.shape[0]}')


In [235]:
# External review data
#file = "../sentiment_analysis_external_review/data/train/5_dataApp_EN_clean_vader.csv"
#file = "../sentiment_analysis_external_review_nbank/data/predict/OUT_sentiment_prediction.csv"

#file = "4_dataApp_EN_2022-09-23.csv"
df_pred_ex = pd.read_csv(REVIEW_FILE_PATH, encoding='utf-16', sep='\t',index_col=False)
df_pred_ex = df_pred_ex[['date','review','review_adj','review_en','sentiment']]
df_pred_ex['date'] = pd.to_datetime(df_pred_ex['date'])
df_pred_ex = df_pred_ex[(df_pred_ex['date']>=DATE_FROM)&(df_pred_ex['date']<=DATE_TO)]

# # Internal review data
# file = "../sentiment_analysis_internal_review/data/train/5_dataApp_EN_clean_vader.csv"
# #file = "4_dataApp_EN_2022-09-23.csv"
# df_pred_in = pd.read_csv(file, index_col=False)
# df_pred_in = df_pred_in[['date','review','review_adj','review_en','label']]
# df_pred_in['date'] = pd.to_datetime(df_pred_in['date'])
# df_pred_in = df_pred_in[df_pred_in['date']>=datetime.datetime(2022,9,1)]

# df_pred = pd.concat([df_pred_ex, df_pred_in])
df_pred = df_pred_ex
print(f'Total review in this month is: {df_pred.shape[0]}')

Total review in this month is: 154


In [236]:
# file = "../sentiment_analysis_external_review_nbank/data/train/5_dataApp_EN_clean_vader.csv"
# #file = "4_dataApp_EN_2022-09-23.csv"
# df_pred = pd.read_csv(PRED_ROOT+file,index_col=False, sep='\t')

df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub(r'[^\w\s,]','', str(x)))
df_pred = df_pred[df_pred['sentiment']==1] # classify to topic for negative review only

print(f'Number of Negative review in this month is: {df_pred.shape[0]}')

df_pred['review_en'] = df_pred['review_en'].apply(lambda x: str(x))
df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('\'ll',' will', x))
df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('don\'t','do not', x))

df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('can\'t','can not', x))
df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('hasn\'t','has not', x))
df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('wasn\'t','was not', x))
df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('won\'t','will not', x))
df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('cant','can not', x))
df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('dont','do not', x))
df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('hadn\'t','had not', x))
df_pred['review_en'] = df_pred['review_en'].apply(lambda x: re.sub('cannot','can not', x))


print(df_pred['review_en'].values[:3])


Number of Negative review in this month is: 100
['can not open on redmi' 'No registration only login'
 'scam customers to take personal information digitally sign contracts online xg neither approve nor send email to cancel records even though it has been reported to an unreliable banking hotline']


### PREDICTION

In [237]:
# # Load Topic Model
# topic_model1 = BERTopic.load("./model/bertopic_model_weights_rl1.h5")

# with open('./model/topic_dict_rl1.yaml') as f:
#     topic_dict1 = yaml.load(f, Loader=SafeLoader)
# #print(topic_dict1)


topic_model1 = BERTopic.load(TOPIC_MODEL_PATH+"bertopic_model_weights.h5")

with open(TOPIC_MODEL_PATH+'topic_dict.yaml') as f:
    topic_dict1 = yaml.load(f, Loader=SafeLoader)
#print(topic_dict1)

In [238]:
# reviews = ['the app is so bad, performance is quite slow',
#            'login fails always, I am unhappy with the app',
#            'supporting weak, waiting so long time to get the response from staff. I will never use the service any more.',
#            'hard to use the app, the UI is not friendly. it makes me confused.',
#            'cheating coupon campaign, i got nothing',
#            'nothing to say about the service',
#            'transfer money quite slow, near one day to get',
#            'no more atm, inconvenient to withdraw. I can\'t any atm nearby.',
#            'card is not recognized by machine. unconvenient',
#            'can not payment online with the card',
#            'wrong fee for credit card. please refund money back to me.',
#            'call but not receive the relevant response from staff',
#            'app get error in view profile information',
#            'money transfer is slow, it takes long time for receiving.']

pred_rs = topic_model1.transform(df_pred['review_en'].values)
df_pred['topic'] = pred_rs[0]

df_pred['topic_content'] = df_pred['topic'].apply(lambda x: '_'.join(topic_dict1[x]) if (topic_dict1[x]!='outlier') else topic_dict1[x])
#df_pred[['review','review_adj','review_en','topic','topic_content']].to_csv(PRED_ROOT+'/ext_review_topic_OUT.csv', encoding='utf-16', index=False, sep='\t')
df_pred.head(1)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

,date,review,review_adj,review_en,sentiment,topic,topic_content
1,2022-09-28 14:22:37,Không mở được trên redmi note8pro,không mở được trên redmi,can not open on redmi,1,-1,outlier


In [239]:
df_topic_lookup = pd.read_csv(TOPIC_MODEL_PATH+"topic_lookup.csv", index_col=False)
gtopic_lookup_dict = dict(zip(df_topic_lookup["topic"], df_topic_lookup["gtopic_content"]))

df_pred['gtopic_content'] = df_pred['topic'].apply(lambda x: gtopic_lookup_dict[x])
df_pred.to_csv("./predict_data/OUTPUT_topic_modeling.csv", index=False, encoding='utf-16', sep='\t')
df_pred.head(1)

,date,review,review_adj,review_en,sentiment,topic,topic_content,gtopic_content
1,2022-09-28 14:22:37,Không mở được trên redmi note8pro,không mở được trên redmi,can not open on redmi,1,-1,outlier,Outlier


In [240]:
# summary topic result 
df_topic_top = df_pred.groupby(['topic','topic_content'])['review'].count().sort_values(ascending=False).reset_index()
print(df_topic_top[:5])
df_topic_top.to_csv(PRED_ROOT+'pred_topic_result_summary.csv', index=False)
topic_arr = df_topic_top['topic'].values
print(f"Successful Topic distribution rate: {(df_topic_top['review'].sum()-df_topic_top.loc[df_topic_top['topic']==-1,'review'].values[0])/df_topic_top['review'].sum()}")

   topic                                      topic_content  review
0     -1                                            outlier      29
1      0  card_call_staff_bank_credit_fee_credit card_ca...       8
2      2  kyc_crashing_keeps crashing_log_app keeps cras...       6
3     15  transfer_transfer money_money_transfer keeps_n...       6
4      4  download_app bad_not download_download app_not...       4
Successful Topic distribution rate: 0.71


In [241]:
# topic_clus1 = [ i for i in df_pred['topic'].unique() if i>=0]

# for i in topic_clus1:
#     print(i,':',topic_dict1[i])

In [242]:
# !pip install --upgrade pip
# !pip install --upgrade numpy
# !pip install --upgrade sentence_transformers
# !conda install -c conda-forge hdbscan --y
# !pip install bokeh
# !pip install --upgrade bertopic[visualization]

#### Reference
https://www.youtube.com/watch?v=fb7LENb9eag
https://github.com/pinecone-io/examples/blob/master/learn/algos-and-libraries/bertopic/06_bertopic_params.ipynb
https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html#word-document-embeddings
